# Introduction

------------------------------------------------

Wildfires can have many causes, which are determined by expert investigators.  Sometimes, the cause of a fire is obvious: an eyewitness account of arsonists leaving the scene of the crime. However it isn't always an open and shut case nor is it always arson. Other causes can be debris burning, equipment use, lightning, or a campfire gone awry. An investigation is always required. Can machine learning algorithms assist investigators of wildfires?  This is the question I set out to explore. 



In order to build a model, data is required; the more data the better. Mercifully, there’s a fairly clean dataset of over a million wildfires found on Kaggle.  The dataset has many variables. However, not all given variables will be used in the model. Furthermore, part of the art of model building entails bringing in extra variables from different datasets.

A very reasonable feature to include in the modeling process is weather. Getting the weather for 1.88 million rows of data was the most challenging part of the project for a novice coder like myself. Trial and error on 1.88 million rows of data will take forever to compute. Fortunately, I thought of clever shortcuts which are detailed in the notebook. 



Modeling is developed iteratively. In other words,  a base model whose measures of veracity are overall improved with sequential modeling decisions, leading to new models.  At the very least, the model should be better than chance. In a binary classification model, accuracy should be better than 50%.  The base model here gives the right answer 61% of the time, and is a simple logistic model without exogenous features.



For evaluating our model simplistically, we can look at accuracy. The number of observations correctly classified over the total possible number of observations is accuracy.  This is also called the True Positive rate.  Further metrics to get a comprehensive picture of model performance are detailed in the notebook.


-----------------------------------------

# Import Necessary Packages and Modules Here

-------------------------

In [162]:
import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from collections import OrderedDict

import datetime

import operator

import requests

import sklearn.neighbors

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE

-------------------------

# Data Preparation For the Feature of Weather

---------------------------------------

## A Function To Get NOAA Stations

NOAA is the National Oceanic and Atmospheric Administration for the USA. I always thought that was a good acronym for an organization concerned with weather.  NOAA sounds like Noah, the protagonist in the Biblcial Flood story, and a flood is an extreme weather event. But I digress.

NOAA has weather stations all across the United States. NOAA also makes its historical weather data free to use. Therefore, NOAA is a natural choice for getting historical weather conditions for a data science project. My objective is to get the historical weather conditions for 1.88 million wildfires from 1992 to 2015. This objective is do-able, but requires breaking it down into smaller tasks. 

First, I need a list of all the weather stations and their geographic coordinates. However this list wasn't available. I would have to create this list myself.  Fortunately, NOAA lets you download a spreadsheet for the weather on a single day in any state.  It's a request I make on the website and only takes a few minutes to process. When my request is processed, I am sent a link to a website, with the cvs file. This is not an API request. It's manually navigating buttons on a website, done fifty times. 

Then I created a function that will quickly visit these websites, change the cvs file into a dataframe, extract the relevant data, and put that into a new dataframe. These urls will be defunct after a time period. For ease of reproducibility, the function saves its results in a cvs file on the computer.   

In [7]:
def NOAA_stations(url):
    
    df = pd.read_csv(url)

    # Backup csv file from the web
    df.to_csv(url[-11:], index=False)

    api_station_name = ['GHCND:' + i for i in df.STATION.values]

    api_station_state = [df.NAME[0][-5:-3] for i in range(len(df))]
    
    api_station_latitude = df.LATITUDE

    api_station_longitude = df.LONGITUDE

    columns = ['name', 'state', 'latitude', 'longitude']

    data = [api_station_name, api_station_state,
            api_station_latitude, api_station_longitude]

    data = dict(zip(columns, data))

    return pd.DataFrame(data)

Backup the function's results

NOAA_stations_df = pd.concat(df_list, axis = 0)
NOAA_stations_df.to_csv('NOAA_stations_df.csv', index = False)

-----------------------------------------------------------------------------------------

##   Import NOAA Weather Stations & Locations

In [ ]:
NOAA_stations_df = pd.read_csv('NOAA_stations_df.csv')
NOAA_stations_df.drop('index', axis = 1, inplace = True)

# The magic of rounding

At this point, we have a list of all the NOAA weather stations in the United States, with their precise geographic coordinates. We can now determine which station is closer to which wildfire. This will be done through trial and error. However, without using any shortcuts it will take too long.  A major shortcut is rounding.  

In our dataset, the coordinates of wildfires is give to several decimal places. However there are way fewer weather stations than wildfire incidents.  We won't actually lose that much information, if we round the wildfires' coordinates to the ones place.  The number of unique locations drops a lot.

Importing the original dataset of wildfires data from Kaggle.

In [ ]:
df = pd.read_csv("wildfires.csv")

In [ ]:
print("The number of observations " ,len(df))

print("The number of unique latitudes " ,len(set(wildfires_coordinates_df.LATITUDE)))

rounding_effects_lat = [len(set(wildfires_coordinates_df.LATITUDE.round(i))) for i in range(9)]

print("How the number of unique latitudes changes with rounding ")
print(rounding_effects_lat)

This is a subset of the original dataset with only the features we need for finding the closest weather stations.

In [ ]:
wildfires_coordinates_df = df[['OBJECTID','STATE','LATITUDE','LONGITUDE']]

In [ ]:
wildfires_coordinates_df['LATITUDE'] = wildfires_coordinates_df.LATITUDE.round()

wildfires_coordinates_df['LONGITUDE'] = wildfires_coordinates_df.LONGITUDE.round()

Now that the latitudes and longitudes are rounded, some geographic coordinates may be repeated. 
In order to find the unique coordinates, we have to look at latitude and longitude together for each observation.
Therefore we will turn them into tuples. 

In [ ]:
lat_long_tups = list(zip(wildfires_coordinates_df.LATITUDE.values ,wildfires_coordinates_df.LONGITUDE.values))

wildfires_coordinates_df['LAT_LONG'] = lat_long_tups

-------------------------------------------------------------------

# Creating a Dictionary: Wildfire coordinates to OrderIDs

We make a dictionary that has the unique tuples and all the objectids assocaited with them.

In [ ]:
values = [wildfires_coordinates_df[wildfires_coordinates_df.LAT_LONG ==
                                   i].OBJECTID.values for i in lat_long_tups_unique]

geo_dict = dict(zip(lat_long_tups_unique, values))

---------------------------------------------------------------

# Creating a Dictionary of OrderIDs to other OrderIDs.

The values in the code below are all the OrderIDs which share a common geographic coordinate for their wildfire. It follows, that a single orderID can act as a key for all the orderIDs, including itself which have the same wildfire coordinates. 

In [ ]:
new_keys = [i[0] for i in values]

orderID_group_dict = dict(zip(new_keys,values))

-----------------------------------------------------------------

# Haversian Distance

Calculating the distance between two points on a sphere requires some special mathematics.  Fortunately there are functions already made to calculate such distances, called Haversian distances.  Calculations with Haversian distances are done in radians. Latitude and longitude must be converted into radians. I adapted the code from this resource on Haversian Distances:   https://codeburst.io/calculate-haversine-distance-between-two-geo-locations-with-python-439186315f1b

In [ ]:
NOAA_stations_df[['lat_radians_Y', 'long_radians_Y']] = (
    np.radians(NOAA_stations_df.loc[:, ['latitude', 'longitude']]))


Places_X = pd.DataFrame({'OBJECTID': [i[0] for i in values],
                         'latitude': [i[0] for i in lat_long_tups_unique],
                         'longitude': [i[-1] for i in lat_long_tups_unique]})


Places_X[['lat_radians_X', 'long_radians_X']] = np.radians(
    Places_X.loc[:, ['latitude', 'longitude']])

dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')

dist_matrix = (dist.pairwise
               (Places_X[['lat_radians_X', 'long_radians_X']],
                NOAA_stations_df[['lat_radians_Y', 'long_radians_Y']])*3959
               )

df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=Places_X['OBJECTID'], 
                 columns=NOAA_stations_df['name'])
)


df_dist_unpv = (
    pd.melt(df_dist_matrix.reset_index(),id_vars='OBJECTID')
)


In [ ]:
k_list = []

station_name_dict = {}

for k, v in orderID_group_dict.items():

   # put a print statement to know something is running
    
    k_list.append(k)
    
    count = len(k_list)
    
    print(count/1232)

    subset = df_dist_unpv.OBJECTID == k
    
    closest_staton_name = df_dist_unpv.loc[(df_dist_unpv['OBJECTID']==k)].sort_values(by = 'value').head(1).name.values[0]

    station_name_dict[k] = closest_staton_name

# Looking between dictionaries

In [ ]:
master_dict = {}

for k,v in station_name_dict.items():

    for key in orderID_group_dict[k]:
        
        master_dict[key] = v

In [ ]:

master_dict = OrderedDict(sorted(master_dict.items()))

# Grand Finale - Making the Closest Weather Station  a Feature

In [ ]:
wildfires_coordinates_df['NOAA'] = list(master_dict.values())

# Converting Julian Dates to Standard Dates

In [ ]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')

wildfires_coordinates_df['DISCOVERY_DATE'] = df['DISCOVERY_DATE']

# Save the DataFrame 

In [ ]:
wildfires_coordinates_df.to_csv('wildfires_coordinates_df', index=False)

----------------------

# Adding a Feature of Month

In [6]:
wildfires_coordinates_df = pd.read_csv('wildfires_coordinates_df')

wildfires_coordinates_df['year'] = pd.DatetimeIndex(
    wildfires_coordinates_df.DISCOVERY_DATE).year

# EDA about Causes of Wildfire

In [304]:
wildfires_df.STAT_CAUSE_DESCR.value_counts(normalize = True)

Debris Burning       0.228150
Miscellaneous        0.172194
Arson                0.149673
Lightning            0.148085
Missing/Undefined    0.088661
Equipment Use        0.078498
Campfire             0.040489
Children             0.032528
Smoking              0.028115
Railroad             0.017791
Powerline            0.007683
Fireworks            0.006116
Structure            0.002019
Name: STAT_CAUSE_DESCR, dtype: float64

# Which  and When

## Which State had the most fires and in which year? 

In [7]:
groups = wildfires_coordinates_df.groupby(['year', 'STATE']).count().OBJECTID

unstack_df = groups.unstack()

unstack_df.fillna(0, inplace=True)

In [8]:
state_dict = dict(zip(list(unstack_df.columns), [max(
    unstack_df[i]) for i in list(unstack_df.columns)]))

max_value = max(state_dict.values())

max_key = max(state_dict.items(), key=operator.itemgetter(1))[0]


print('The most wildires in a single year was ' +
      str(round(max_value)) + ',' + ' occurring in ' + str(max_key) + '.')

The most wildires in a single year was 19453, occurring in TX.


In [10]:
Texas_df = wildfires_coordinates_df[wildfires_coordinates_df.STATE == 'TX']

In [11]:
year_dict = dict(zip(Texas_df.groupby(['year']).count(
).OBJECTID.index, Texas_df.groupby(['year']).count().OBJECTID.values))

max_value = max(year_dict.values())

max_key = max(year_dict.items(), key=operator.itemgetter(1))[0]

print('The year with ' + str(max_value) +
      ' wildfires was ' + str(max_key) + '.')

The year with 19453 wildfires was 2011.


In [14]:
Texas_2011_df = Texas_df[Texas_df.year == 2011]

---------------------------------------------------

# Get Features & Target

## API Call for Weather Tempertures

### API function

In [18]:
def get_weather(datasetid, stationid, startdate, enddate, datatypeid ,limit, token):

    token = {'token': token}

    params = 'datasetid=' + datasetid + '&' + 'stationid=' + stationid + '&' + \
        'startdate=' + str(startdate) + '&' + 'enddate=' + \
        str(enddate) + '&' + 'datatypeid=' + datatypeid + '&' + 'limit=' + str(limit)

    r = requests.get(base_url, params=params, headers=token)

    print("Request status code: "+ str(r.status_code))

    return json.loads(r.text)

## API Weather data

In [19]:
station_list = set(Texas_2011_df.NOAA)

station_list_dict = dict(enumerate(station_list))

In [30]:
token = 'wtvrEYejcUwYDltcGRmzjkkHkxEhQfor'

base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"

startdate = '2011-01-01'

enddate = '2011-12-31'

datatypeid = 'TMAX'

results = [get_weather('GHCND', i, startdate, enddate, datatypeid , 1000 , token ) for i in station_list] 

Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200
Request status code: 200


In [31]:
results

[{},
 {'metadata': {'resultset': {'offset': 1, 'count': 358, 'limit': 1000}},
  'results': [{'date': '2011-01-01T00:00:00',
    'datatype': 'TMAX',
    'station': 'GHCND:USC00297867',
    'attributes': ',,7,1800',
    'value': 28},
   {'date': '2011-01-02T00:00:00',
    'datatype': 'TMAX',
    'station': 'GHCND:USC00297867',
    'attributes': ',,7,1800',
    'value': 117},
   {'date': '2011-01-03T00:00:00',
    'datatype': 'TMAX',
    'station': 'GHCND:USC00297867',
    'attributes': ',,7,1800',
    'value': 167},
   {'date': '2011-01-04T00:00:00',
    'datatype': 'TMAX',
    'station': 'GHCND:USC00297867',
    'attributes': ',,7,1800',
    'value': 128},
   {'date': '2011-01-05T00:00:00',
    'datatype': 'TMAX',
    'station': 'GHCND:USC00297867',
    'attributes': ',,7,1800',
    'value': 161},
   {'date': '2011-01-06T00:00:00',
    'datatype': 'TMAX',
    'station': 'GHCND:USC00297867',
    'attributes': ',,7,1800',
    'value': 139},
   {'date': '2011-01-07T00:00:00',
    'datatype

## Subset dataframe based on available weather station data from the API call

In [32]:
station_result_dict = dict(
    enumerate([len(results[i]) != 0 for i in range(len(station_list))]))

conditions = []

for k, v in station_list_dict.items():
    if station_result_dict[k] == True:
        conditions.append(v)

In [34]:
Texas_2011_df_ready = pd.concat([Texas_2011_df[Texas_2011_df.NOAA == i] for i in conditions], axis = 0)

## Turn api results into a dataframe

In [37]:
station_result_dict

{0: False,
 1: True,
 2: False,
 3: False,
 4: False,
 5: False,
 6: False,
 7: False,
 8: True,
 9: True,
 10: False,
 11: False,
 12: True,
 13: True,
 14: False,
 15: False,
 16: True,
 17: True,
 18: True,
 19: False,
 20: True,
 21: False,
 22: True,
 23: True,
 24: False,
 25: False,
 26: False,
 27: False,
 28: True,
 29: True,
 30: False}

In [38]:
filtered = [k for k in station_result_dict.keys() if station_result_dict[k] == True]

filtered

[1, 8, 9, 12, 13, 16, 17, 18, 20, 22, 23, 28, 29]

In [39]:





api_result_df_list = []

for i in filtered:

    api_result_df = pd.DataFrame(results[i]['results'])

    api_result_df.drop('attributes', axis = 1, inplace = True)

    api_result_df['date'] = pd.to_datetime(api_result_df['date'], format = '%Y-%m-%d')
    
    api_result_df_list.append(api_result_df)

api_result_df = pd.concat(api_result_df_list)

In [40]:
# Make a nested dictionary to make it easier to enter Temperture data for modeling
# first key is noaa station the values are the keys of dates, whose values are the temperture readings.

keys = list(api_result_df.station)

feature_making_dict = {}

for i in keys:
    
    feature_making_dict[i] = ''

    pre_dict_df = api_result_df[api_result_df.station ==
                            i][['date', 'value']]
    dict2 = dict(zip(pre_dict_df.date,pre_dict_df.value))
    
    feature_making_dict[i] = dict2

feature_making_dict

{'GHCND:USC00297867': {Timestamp('2011-01-01 00:00:00'): 28,
  Timestamp('2011-01-02 00:00:00'): 117,
  Timestamp('2011-01-03 00:00:00'): 167,
  Timestamp('2011-01-04 00:00:00'): 128,
  Timestamp('2011-01-05 00:00:00'): 161,
  Timestamp('2011-01-06 00:00:00'): 139,
  Timestamp('2011-01-07 00:00:00'): 133,
  Timestamp('2011-01-08 00:00:00'): 150,
  Timestamp('2011-01-09 00:00:00'): 133,
  Timestamp('2011-01-10 00:00:00'): 11,
  Timestamp('2011-01-11 00:00:00'): 39,
  Timestamp('2011-01-12 00:00:00'): 39,
  Timestamp('2011-01-13 00:00:00'): 139,
  Timestamp('2011-01-14 00:00:00'): 150,
  Timestamp('2011-01-15 00:00:00'): 167,
  Timestamp('2011-01-16 00:00:00'): 172,
  Timestamp('2011-01-17 00:00:00'): 239,
  Timestamp('2011-01-18 00:00:00'): 172,
  Timestamp('2011-01-19 00:00:00'): 206,
  Timestamp('2011-01-20 00:00:00'): 56,
  Timestamp('2011-01-21 00:00:00'): 122,
  Timestamp('2011-01-22 00:00:00'): 206,
  Timestamp('2011-01-23 00:00:00'): 106,
  Timestamp('2011-01-24 00:00:00'): 94,
 

### Creating the Temperture Feature in the data

I need to lookup keys in a nested dictionary.

In [41]:
def df_row(row_number):
    return Texas_2011_df_ready[['NOAA', 'DISCOVERY_DATE']].iloc[row_number, ]

In [75]:
# this gives me the two keys in the nested dictionary...

TMAX = [feature_making_dict.get(df_row(i)[0]).get(pd.to_datetime(df_row(i)[1])) for i in range(len(Texas_2011_df_ready))]

In [77]:
Texas_2011_df_ready['TMAX'] = TMAX

In [81]:
df_features = Texas_2011_df_ready

## Getting the Fire Cause, Day of the Year Feature, Time of Fire From Original Dateset

In [78]:
wildfires_df = pd.read_csv('wildfires.csv')

/Users/jonathansilverman/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [82]:
objectID_needed = df_features.OBJECTID.values

fire_cause_dict = dict(zip(wildfires_df.OBJECTID.values, wildfires_df.STAT_CAUSE_DESCR))

DOY_dict = dict(zip(wildfires_df.OBJECTID.values, wildfires_df.DISCOVERY_DOY))

TIME_dict = dict(zip(wildfires_df.OBJECTID.values, wildfires_df.DISCOVERY_TIME))

df_features['STAT_CAUSE_DESCR'] = [fire_cause_dict.get(i) for i in objectID_needed]

df_features['DISCOVERY_DOY'] = [DOY_dict.get(i) for i in objectID_needed]

df_features['DISCOVERY_TIME'] = [TIME_dict.get(i) for i in objectID_needed]

In [83]:
df_features.head() 

,OBJECTID,STATE,LATITUDE,LONGITUDE,LAT_LONG,NOAA,DISCOVERY_DATE,year,TMAX,STAT_CAUSE_DESCR,DISCOVERY_DOY,DISCOVERY_TIME
1459871,1459872,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-05-24,2011,294.0,Miscellaneous,144,1436.0
1525270,1525271,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-05-11,2011,272.0,Lightning,131,1430.0
1525565,1525566,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-06-12,2011,378.0,Lightning,163,2200.0
1525932,1525933,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-07-12,2011,361.0,Lightning,193,1735.0
1550976,1550977,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-02-20,2011,183.0,Equipment Use,51,NaN


In [85]:
data_df = df_features[df_features.STAT_CAUSE_DESCR != 'Missing/Undefined']

# Logistic Classifier

### Class Imbalances

In [86]:
data_df.STAT_CAUSE_DESCR.value_counts(normalize = True)


Miscellaneous     0.285734
Debris Burning    0.265682
Equipment Use     0.163257
Lightning         0.086167
Powerline         0.078851
Arson             0.059748
Smoking           0.031161
Campfire          0.012058
Children          0.011652
Railroad          0.005419
Fireworks         0.000271
Name: STAT_CAUSE_DESCR, dtype: float64

In [88]:
print('Debris Burning: ',sum(data_df.STAT_CAUSE_DESCR == 'Debris Burning')/len(data_df))

print('not Debris Burning: ',sum(data_df.STAT_CAUSE_DESCR != 'Debris Burning')/len(data_df))

Debris Burning:  0.2656821568893104
not Debris Burning:  0.7343178431106896


## Single Class

Was it debris burning, or was it not debris burning?

In [108]:
target = data_df.STAT_CAUSE_DESCR == 'Debris Burning'

data_df['target'] = [int(i) for i in target.values]

<ipython-input-108-eb813113346c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['target'] = [int(i) for i in target.values]


In [120]:
data_df['month'] = [i.month for i in pd.to_datetime(data_df['DISCOVERY_DATE'].values)]

<ipython-input-120-b9ce9b5584a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['month'] = [i.month for i in pd.to_datetime(data_df['DISCOVERY_DATE'].values)]


In [121]:
data_df.head()

,OBJECTID,STATE,LATITUDE,LONGITUDE,LAT_LONG,NOAA,DISCOVERY_DATE,year,TMAX,STAT_CAUSE_DESCR,DISCOVERY_DOY,DISCOVERY_TIME,target,month
1459871,1459872,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-05-24,2011,294.0,Miscellaneous,144,1436.0,0,5
1525270,1525271,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-05-11,2011,272.0,Lightning,131,1430.0,0,5
1525565,1525566,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-06-12,2011,378.0,Lightning,163,2200.0,0,6
1525932,1525933,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-07-12,2011,361.0,Lightning,193,1735.0,0,7
1550976,1550977,TX,35.0,-103.0,"(35.0, -103.0)",GHCND:USC00297867,2011-02-20,2011,183.0,Equipment Use,51,NaN,0,2


### Baseline Model (without exogenous features)

#### y

In [181]:
y = data_df.target

#### X

####  Continous Features

In [182]:
continous = ['LATITUDE', 'LONGITUDE', 'DISCOVERY_DOY', 'DISCOVERY_TIME']

#### Categorical Features

In [183]:
categorical = ['month']

X_categorical = pd.get_dummies(data_df[categorical], prefix='month')

###  Normalize the data

In [184]:
X_continous = data_df[continous].fillna(value=0)

for column in X_continous.columns:

    # Subtract the minimum and divide by the range forcing a scale of 0 to 1 for each feature

    X_continous[column] = (X_continous[column] - min(X_continous[column])) / \
        (max(X_continous[column]) - min(X_continous[column]))

In [185]:
### Concatenate continous and categorical dataframes

In [186]:
X = pd.concat([X_continous,X_categorical], axis = 1)

##  Train-Test Split

In [187]:
#deal with imbalances

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

smote = SMOTE()

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train) 

## Fit the model

In [188]:
logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')

model_log = logreg.fit(X_train_resampled, y_train_resampled)

 ## Predict

In [189]:
y_hat_train = model_log.predict(X_train)

y_hat_test = model_log.predict(X_test)

## How many times was the classifier correct on the training set?

In [190]:
residuals = np.abs(y_train - y_hat_train)


print(pd.Series(residuals).value_counts(normalize=True))

0    3501
1    2034
Name: target, dtype: int64
0    0.63252
1    0.36748
Name: target, dtype: float64


## How many times was the classifier correct on the testing set?

In [191]:
residuals = np.abs(y_test - y_hat_test)


print(pd.Series(residuals).value_counts(normalize=True))

0    1142
1     704
Name: target, dtype: int64
0    0.618635
1    0.381365
Name: target, dtype: float64


## Model with One Exogenous Feature

In [195]:
continous = ['LATITUDE', 'LONGITUDE',
             'DISCOVERY_DOY', 'DISCOVERY_TIME', 'TMAX']

X_continous = data_df[continous].fillna(value=0)

for column in X_continous.columns:

    X_continous[column] = (X_continous[column] - min(X_continous[column])) / \
        (max(X_continous[column]) - min(X_continous[column]))

X = pd.concat([X_continous, X_categorical], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

smote = SMOTE()

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')

model_log = logreg.fit(X_train_resampled, y_train_resampled)

y_hat_train = model_log.predict(X_train)

y_hat_test = model_log.predict(X_test)


residuals = np.abs(y_train - y_hat_train)


print(pd.Series(residuals).value_counts(normalize=True))

print('\n')

residuals = np.abs(y_test - y_hat_test)


print(pd.Series(residuals).value_counts(normalize=True))

0    0.69449
1    0.30551
Name: target, dtype: float64


0    0.68039
1    0.31961
Name: target, dtype: float64
